In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
data = pd.read_csv("./Churn_Modelling.csv")

In [3]:
X = data.iloc[:,3:-1].values
y = data.iloc[:,-1].values

In [4]:
le = LabelEncoder()

In [5]:
X[:,2] = le.fit_transform(X[:, 2])

In [6]:
# One Hot Encoding for the Geography
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [7]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [8]:
sc = StandardScaler()

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [10]:
X_train = sc.fit_transform(X_train)

In [11]:
X_test = sc.transform(X_test)

In [12]:
X_test

array([[-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
         0.9687384 ,  1.61085707],
       [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
        -1.03227043,  0.49587037],
       [-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
         0.9687384 , -0.42478674],
       ...,
       [-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
        -1.03227043,  0.71888467],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
         0.9687384 , -1.54507805],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
        -1.03227043,  1.61255917]])

In [13]:
X_train = torch.tensor(X_train)

In [14]:
X_test = torch.tensor(X_test)

In [15]:
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

In [16]:
y_train = torch.unsqueeze(y_train, 1).float()
y_test = torch.unsqueeze(y_test, 1).float()

In [17]:
print("X Training Model Shape :" ,X_train.shape)
print("X Training Model Shape :" ,X_test.shape)

X Training Model Shape : torch.Size([8000, 12])
X Training Model Shape : torch.Size([2000, 12])


In [18]:
print("y Training Model Shape :" ,y_train.shape)
print("y Training Model Shape :" ,y_test.shape)

y Training Model Shape : torch.Size([8000, 1])
y Training Model Shape : torch.Size([2000, 1])


In [58]:
class Model(nn.Module):
    def __init__(self, no_input, no_hidden1, no_hidden2, no_hidden3, no_output):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(no_input, no_hidden1)
        self.fc2 = nn.Linear(no_hidden1,no_hidden2)
        self.fc3 = nn.Linear(no_hidden2,no_hidden3)
        self.fc4 = nn.Linear(no_hidden3,no_output)
        # Defining Activation Function
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

In [59]:
net = Model(X_train.shape[1], 50,50,50, 1)

In [60]:
# Loss Function
bce = torch.nn.BCELoss(reduction="mean")

In [61]:
# Optimizer (We will use SGD)
optimizer = torch.optim.Adam(net.parameters())

In [62]:
class Dataset(Dataset):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

dataset = Dataset(X_train, y_train)
train_loader = DataLoader(dataset=dataset,
          batch_size=100,
          shuffle=True)

In [63]:
# Training The Data
epochs = 100
for epoch in range(epochs):
    for inputs,labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        outputs = net(inputs)
        loss = bce(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    output = (outputs > 0.5).float() # returns [1.,0.,0.,1.,...]
    accuracy = (output == labels).float().mean() # Check if they are equal and returns 1 if equal, 0 not equal
    # Another way of calculating is 
    #   (output == labels).sum() / output.shape[0]

    # Print Statistics
    print("Epoch {}/{}, Loss : {:.3f}, Accuracy: {:.3f}".format(epoch+1, epochs, loss, accuracy))

Epoch 1/100, Loss : 0.320, Accuracy: 0.830
Epoch 2/100, Loss : 0.330, Accuracy: 0.880
Epoch 3/100, Loss : 0.387, Accuracy: 0.820
Epoch 4/100, Loss : 0.392, Accuracy: 0.840
Epoch 5/100, Loss : 0.404, Accuracy: 0.820
Epoch 6/100, Loss : 0.294, Accuracy: 0.890
Epoch 7/100, Loss : 0.275, Accuracy: 0.890
Epoch 8/100, Loss : 0.341, Accuracy: 0.910
Epoch 9/100, Loss : 0.242, Accuracy: 0.920
Epoch 10/100, Loss : 0.466, Accuracy: 0.820
Epoch 11/100, Loss : 0.346, Accuracy: 0.860
Epoch 12/100, Loss : 0.344, Accuracy: 0.840
Epoch 13/100, Loss : 0.316, Accuracy: 0.910
Epoch 14/100, Loss : 0.311, Accuracy: 0.890
Epoch 15/100, Loss : 0.379, Accuracy: 0.870
Epoch 16/100, Loss : 0.427, Accuracy: 0.840
Epoch 17/100, Loss : 0.365, Accuracy: 0.790
Epoch 18/100, Loss : 0.340, Accuracy: 0.890
Epoch 19/100, Loss : 0.308, Accuracy: 0.860
Epoch 20/100, Loss : 0.292, Accuracy: 0.860
Epoch 21/100, Loss : 0.274, Accuracy: 0.880
Epoch 22/100, Loss : 0.268, Accuracy: 0.870
Epoch 23/100, Loss : 0.209, Accuracy: 0.9

In [64]:
dataset2 = Dataset(X_test, y_test)
corr = 0
for i, inputs in enumerate(X_test):
    output = net(inputs.float())
    if (output > .5 and y_test[i] == 1) or (output < .5 and y_test[i] == 0):
        corr += 1

print("Accuracy : {}".format(corr/y_test.shape[0]))

Accuracy : 0.8505
